In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)

In [2]:
sample = " if you want you"
idx2char = list(set(sample)) # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)} # char -> index

In [3]:
# hyper parameters
dic_size = len(char2idx) # RNN input size (one hot size)
hidden_size = len(char2idx) # RNN output size
num_classes = len(char2idx) # final output size(RNN or softmax, etc.)
batch_size = 1 # one sample data, one batch
sequence_length = len(sample) - 1 # number of lstm rollings (unit #)
learning_rate = 0.1

In [4]:
sample_idx = [char2idx[c] for c in sample] # char to index
x_data = [sample_idx[:-1]] # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]  # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length]) # X data
Y = tf.placeholder(tf.int32, [None, sequence_length]) # Y label

In [5]:
x_one_hot = tf.one_hot(X, num_classes) # one hot 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(30):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})
        
        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print(i, "loss", l, "prediction: ", ''.join(result_str))

0 loss 2.29793 prediction:  y  y     oooooo
1 loss 2.1150959 prediction:  y         o  o 
2 loss 1.9880894 prediction:  y  y uuyyyo you
3 loss 1.7465378 prediction:  y  youuyyyo you
4 loss 1.513832 prediction:  y  youuwano you
5 loss 1.2602837 prediction:  y  youuwant you
6 loss 0.9927257 prediction:  y  youuwant you
7 loss 0.77193797 prediction:  yf you want you
8 loss 0.57580054 prediction:  yf you want you
9 loss 0.4221878 prediction:  yf you want you
10 loss 0.30995324 prediction:  if you want you
11 loss 0.22417742 prediction:  if you want you
12 loss 0.1586844 prediction:  if you want you
13 loss 0.11175729 prediction:  if you want you
14 loss 0.079212435 prediction:  if you want you
15 loss 0.05726131 prediction:  if you want you
16 loss 0.04273827 prediction:  if you want you
17 loss 0.03277855 prediction:  if you want you
18 loss 0.025697442 prediction:  if you want you
19 loss 0.020463329 prediction:  if you want you
20 loss 0.016385537 prediction:  if you want you
21 loss 0.